In [6]:
import json
import os
import yaml
import numpy as np
import sys
from seq2seq.data import vocab

In [21]:
%%bash
s=vie
t=eng
use_lex=""
source ../../../config.sh $s $t
train_src=$trans_dir/train/${src_label}_raw${use_lex}.$s.train.$yrv
dev_src=$trans_dir/dev/${src_label}_raw.$s.dev.$yrv
train_tgt=$trans_dir/train/${ref_label}_raw${use_lex}.$t.train.$yrv
dev_tgt=$trans_dir/dev/${ref_label}_raw.$t.dev.$yrv
echo "train_src: $train_src" > config_local2.yml
echo "dev_src: $dev_src" >> config_local2.yml
echo "train_tgt: $train_tgt" >> config_local2.yml
echo "dev_tgt: $dev_tgt" >> config_local2.yml
echo "bilingual_lexicon: $bilingual_lexicon" >> config_local2.yml

In [22]:
with open("config_local2.yml") as f:
    config = yaml.load(f)
train_src = config["train_src"]
dev_src = config["dev_src"]
train_tgt = config["train_tgt"]
dev_tgt = config["dev_tgt"]
bilingual_lexicon = config["bilingual_lexicon"]

In [23]:
def pad_to_lex_cap(lex_list, lex_cap):
    return np.tile(lex_list, int(lex_cap/len(lex_list))+1)[:lex_cap]
lexicon_dict = vocab.get_lexicon_dict(bilingual_lexicon)

num_tok_src_hit = 0 
num_tok_src = 0
set_tok_src_hit = set()
set_tok_src = set()

with open(train_src) as f_src, open(train_tgt) as f_tgt:
    for l_src in f_src:
        l_src = l_src.strip().split()
        l_tgt = set(f_tgt.readline().strip().split())
        
        for tok_src in l_src:
            num_tok_src += 1
            set_tok_src.add(tok_src)
            if tok_src in l_tgt:
                num_tok_src_hit += 1
                set_tok_src_hit.add(tok_src)
            else:
                if tok_src in lexicon_dict:
                    for tok_src_trans in lexicon_dict[tok_src]:
                        if tok_src_trans in l_tgt:
                            num_tok_src_hit += 1
                            set_tok_src_hit.add(tok_src)
print("num_tok_src_hit: {0}".format(num_tok_src_hit))
print("num_tok_src: {0}".format(num_tok_src))      
print(str(num_tok_src_hit*1.0/num_tok_src))
print("set_tok_src_hit: {0}".format(len(set_tok_src_hit)))        
print("set_tok_src: {0}".format(len(set_tok_src)))
print(str(len(set_tok_src_hit)*1.0/len(set_tok_src)))

num_tok_src_hit: 303983
num_tok_src: 443095
0.6860447533824575
set_tok_src_hit: 10292
set_tok_src: 17632
0.5837114337568058
